# Deploy SD2.1 to Inferentia2 + SageMaker + HF Optimum Neuron + SageMaker SSH Helper

**SageMaker Studio Kernel**: Python 3 (PyTorch 1.13 Python 3.9 CPU Optimized)  
**Instance**: ml.t3.medium

SageMaker SSH Helper is the "army-knife" library that helps you to securely connect to training jobs, processing jobs, batch inference jobs and realtime inference endpoints as well as SageMaker Studio Notebooks and SageMaker Notebook Instances for fast interactive experimentation, remote debugging, and advanced troubleshooting.


Learn more about remote development and debugging with SageMaker SSH Helper in the GitHub repository: [https://github.com/aws-samples/sagemaker-ssh-helper](https://github.com/aws-samples/sagemaker-ssh-helper)

In this example we use SageMaker SSH Helper to connect to remote endpoint instance running on ml.inf2 instance.

## 1) Update SageMaker SDK

In [17]:
%pip install -U sagemaker
%pip install matplotlib


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
# See https://github.com/aws-samples/sagemaker-ssh-helper
%pip install sagemaker-ssh-helper


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 2) Initialize session

In [19]:
import os
os.environ['AWS_DEFAULT_REGION'] = 'us-east-2'

import sagemaker

print(sagemaker.__version__)
if not sagemaker.__version__ >= "2.146.0": print("You need to upgrade or restart the kernel if you already upgraded")

sess = sagemaker.session.Session()
role = sagemaker.get_execution_role()

bucket = sess.default_bucket()
region = sess.boto_region_name

os.makedirs("src", exist_ok=True)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {bucket}")
print(f"sagemaker session region: {region}")

2.200.1
sagemaker role arn: arn:aws:iam::590694872974:role/Admin
sagemaker bucket: sagemaker-us-east-2-590694872974
sagemaker session region: us-east-2


## 3) Create artifacts to compile & run the model

### 3.1) Dependencies file

In [20]:
!pygmentize src/requirements.txt

evaluate==0.4.0
diffusers==0.21.4
accelerate==0.23.0
scikit-learn==1.3.0
transformers==4.33.1
optimum-neuron==0.0.12
sagemaker-ssh-helper


### 3.2) Python script for compiling and deploying the model

This script will download model weights from HF, compile each module to inf2 and save the compiled artifacts to S3

The envvar **NEURON_RT_NUM_CORES** controls how many NeuronCores are allocated per process. SageMaker can launch multiple processes in just one Endpoint. It means you can increase throughput of your endpoint by deploying your model to a larger instance (inf2.24xlarge or inf2.48xlarge). On an inf2.24xlarge, for instance, SageMaker can deploy 6 copies of the same model in parallel to serve 6 simultaneous clients. This is what we'll do in the next steps.

In [21]:
!pygmentize src/compile.py

# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: MIT-0

import os
os.environ['NEURON_RT_NUM_CORES'] = '2'
import io
import json
import time
import shutil
import argparse
from filelock import FileLock
from optimum.neuron import NeuronStableDiffusionPipeline

lock_path = '/tmp/new_packages.lock'
lock = FileLock(lock_path)


def model_fn(model_dir, context=None):
    print("Waiting for the lock acquire...")    
    lock.acquire()
    print("Loading model...")
    t = time.time()    
    model = NeuronStableDiffusionPipeline.from_pretrained(model_dir,  device_ids=[0, 1])
    print(f"Model loaded. Elapsed: {time.time()-t}s")
    lock.release()
    return model

    
def input_fn(request_body, request_content_type, context=None):
    if request_content_type == 'application/json':
        req = json.loads(request_body)
        prompt = req.get('prompt')
        num_inference_steps = req.get('num_inference_steps', 50)
        guidance_scale = req.

## 4) SageMaker (training) Job that will download and compile the model

**ATTENTION**: You need to run this step only once. Then, you can deploy the compiled model as many times as you need.

In [22]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point="compile.py", # Specify your train script
    source_dir="src",
    role=role,
    sagemaker_session=sess,    
    instance_count=1,
    instance_type='ml.trn1.2xlarge',
    output_path=f"s3://{bucket}/output",
    disable_profiler=True,
    disable_output_compression=True,

    image_uri=f"763104351884.dkr.ecr.{region}.amazonaws.com/pytorch-training-neuronx:1.13.1-neuronx-py310-sdk2.13.2-ubuntu20.04",
    
    volume_size = 128,
    hyperparameters={
        "model_id": "stabilityai/stable-diffusion-2-1-base",
    }
)
estimator.framework_version = '1.13.1' # workaround when using image_uri

In [23]:
# See https://github.com/aws-samples/sagemaker-ssh-helper
from sagemaker_ssh_helper.wrapper import SSHEstimatorWrapper 
ssh_wrapper = SSHEstimatorWrapper.create(estimator, connection_wait_time_seconds=600) 

In [24]:
# this takes ~66.16mins on a trn1.32xlarge
estimator.fit(wait=False)

Using provided s3_resource


INFO:sagemaker:Creating training-job with name: pytorch-training-neuronx-2023-12-15-15-39-11-370


In [ ]:
ssh_wrapper.print_ssh_info()
ssh_wrapper.wait_training_job()

INFO:sagemaker-ssh-helper:Resolving training instance IDs through SSM tags
INFO:sagemaker-ssh-helper:Remote training logs are at https://us-east-2.console.aws.amazon.com/cloudwatch/home?region=us-east-2#logsV2:log-groups/log-group/$252Faws$252Fsagemaker$252FTrainingJobs$3FlogStreamNameFilter$3Dpytorch-training-neuronx-2023-12-15-15-39-11-370$252F
INFO:sagemaker-ssh-helper:Estimator metadata is at https://us-east-2.console.aws.amazon.com/sagemaker/home?region=us-east-2#/jobs/pytorch-training-neuronx-2023-12-15-15-39-11-370
INFO:sagemaker-ssh-helper:SSMManager:Querying SSM instance IDs for training job pytorch-training-neuronx-2023-12-15-15-39-11-370, expected instance count = 1
INFO:sagemaker-ssh-helper:SSMManager:Using AWS Region: us-east-2


Remote training logs are at https://us-east-2.console.aws.amazon.com/cloudwatch/home?region=us-east-2#logsV2:log-groups/log-group/$252Faws$252Fsagemaker$252FTrainingJobs$3FlogStreamNameFilter$3Dpytorch-training-neuronx-2023-12-15-15-39-11-370$252F
Training job metadata is at https://us-east-2.console.aws.amazon.com/sagemaker/home?region=us-east-2#/jobs/pytorch-training-neuronx-2023-12-15-15-39-11-370


INFO:sagemaker-ssh-helper:SSMManager:No instance IDs found. Seconds left before time out: 900
INFO:sagemaker-ssh-helper:SSMManager:No instance IDs found. Seconds left before time out: 890
INFO:sagemaker-ssh-helper:SSMManager:No instance IDs found. Seconds left before time out: 880
INFO:sagemaker-ssh-helper:SSMManager:No instance IDs found. Seconds left before time out: 870
INFO:sagemaker-ssh-helper:SSMManager:No instance IDs found. Seconds left before time out: 860
INFO:sagemaker-ssh-helper:SSMManager:No instance IDs found. Seconds left before time out: 850
INFO:sagemaker-ssh-helper:SSMManager:No instance IDs found. Seconds left before time out: 840
INFO:sagemaker-ssh-helper:SSMManager:No instance IDs found. Seconds left before time out: 830
INFO:sagemaker-ssh-helper:SSMManager:No instance IDs found. Seconds left before time out: 820
INFO:sagemaker-ssh-helper:SSMManager:No instance IDs found. Seconds left before time out: 810


In [ ]:
print("""
If you decide to run this notebook again, you don't need to re-compile the model.
Just keep the following path and use it to deploy the model next time.
""")
print(estimator.model_data)

## 5) Deploy the compiled model to a SageMaker endpoint on inf2
SageMaker can launch multiple workers, depending on the size of the Inf2 instance. A worker is a standalone Python process that manages one copy of the model. SageMaker puts a load balancer on top of all these processes and distributes the load automatically for your clients. It means that you can increase throughput by launching multiple workers, which serve different clients in parallel.

For instance. If you deploy the model to a **ml.inf2.48xlarge**, SageMaker can launch 12 workers with 12 copies of the model. This instance has 24 cores and each copy of the model utilizes 2 cores. Then, you can have 12 simultaneous clients invoking the endpoint and being served at the same time.

In [ ]:
import logging
from sagemaker.utils import name_from_base
from sagemaker.pytorch.model import PyTorchModel

# depending on the inf2 instance you deploy the model you'll have more or less accelerators
# we'll ask SageMaker to launch 1 worker per core

instance_type_idx = 1
instance_types = ['ml.inf2.8xlarge', 'ml.inf2.24xlarge','ml.inf2.48xlarge']
num_cores = [2,12,24]
num_workers = num_cores[instance_type_idx]//2

model_data = estimator.model_data
#model_data = {'S3DataSource': {'S3Uri': 's3://sagemaker-us-east-2-555555555555/output/pytorch-training-neuronx-2023-11-29-15-32-12-256/output/model/', 'S3DataType': 'S3Prefix', 'CompressionType': 'None'}}


print(f"Instance type: {instance_types[instance_type_idx]}. Num SM workers: {num_workers}")
pytorch_model = PyTorchModel(
    image_uri=f"763104351884.dkr.ecr.{region}.amazonaws.com/pytorch-inference-neuronx:1.13.1-neuronx-py310-sdk2.14.1-ubuntu20.04",
    model_data=model_data,
    role=role,    
    name=name_from_base('sd'),
    sagemaker_session=sess,
    container_log_level=logging.DEBUG,
    model_server_workers=num_workers,
    framework_version="1.13.1",
    env = {
        'SAGEMAKER_MODEL_SERVER_TIMEOUT' : '3600',
    },
    # for production, it is important to define vpc_config and use a vpc_endpoint
    #vpc_config={
    #    'Subnets': ['<SUBNET1>', '<SUBNET2>'],
    #    'SecurityGroupIds': ['<SECURITYGROUP1>', '<DEFAULTSECURITYGROUP>']
    #}
)
pytorch_model._is_compiled_model = True

In [ ]:
# See https://github.com/aws-samples/sagemaker-ssh-helper
from sagemaker_ssh_helper.wrapper import SSHModelWrapper
ssh_wrapper = SSHModelWrapper.create(pytorch_model, connection_wait_time_seconds=0)

In [ ]:
predictor = pytorch_model.deploy(
    initial_instance_count=1,
    instance_type=instance_types[instance_type_idx],
    model_data_download_timeout=600, # it takes some time to download all the artifacts and load the model
    container_startup_health_check_timeout=600
)

In [ ]:
# See https://github.com/aws-samples/sagemaker-ssh-helper
ssh_wrapper.print_ssh_info()

## 6) Run a simple test to check the endpoint

In [ ]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import BytesDeserializer
predictor.serializer = JSONSerializer()
predictor.deserializer = BytesDeserializer(accept='image/jpeg')

In [ ]:
%matplotlib inline

import io
import time
from PIL import Image
import matplotlib.pyplot as plt
from multiprocessing.pool import ThreadPool

# adjust this number according to the instance size and number of workers
num_clients = num_workers
input_req = {
    "prompt": "A giant crocodile crawling through the jungles with people riding on his top. The crocodile the size of a truck, with his paws serving as the truck wheels",
    # more info about these 2 params here: https://huggingface.co/blog/stable_diffusion
    "num_inference_steps": 25,
    "guidance_scale": 7.5
}
input_reqs = [input_req] * num_clients

def predict(req):    
    data = predictor.predict(req)
    return data

print("The model latency per prediction in BF16 is ~2.01s (50 iterations) ~1.26s (25 iterations).")
print("The time you'll get here also includes IO (data transfer in and out). You can reduce that by defining a VPC-endpoint in the PyTorchModel above.")
with ThreadPool(num_clients) as p:
    t = time.time()
    data = p.map(predict, input_reqs)
    print(f"{len(data)} images generated in {(time.time()-t):0.2f}s.")

fig = plt.figure(figsize=(10, 10))
columns = 2
rows = 3
j = 0
for i in range(1, columns*rows +1):    
    fig.add_subplot(rows, columns, i)
    plt.imshow(Image.open(io.BytesIO(data[j])))
    j += 1
plt.show()

### Delete the endpoint (manually or scheduled)

In [ ]:
import time

last_cell_timestamp = time.time()

If you're running the notebook with "Run All Cells" command, the above cell and the below cell will be executed automatically one by one and the endpoint will stay active for you to experiment with. It will incur additional hourly charges.

However, if you run the below cell manually again after some time depending on the `grace_period_seconds` variable, the endpoint will be gracefully deleted and no further charges will occur.

If you forget to delete the endpoint manually, but will keep the notebook kernel instance up and running, the endpoint will be deleted by the background thread depending on the `auto_delete_hours` variable.

In [ ]:
import threading
import botocore.exceptions

seconds_since_last_cell_timestamp = int(time.time() - last_cell_timestamp)
grace_period_seconds = 30
auto_delete_hours = 8.0

def delete_endpoint():
    try:
        predictor.delete_endpoint(delete_endpoint_config=False)
    except botocore.exceptions.ClientError as e:
        # Most likely, already deleted
        print(e)
        pass
    print("Endpoint has been deleted.")
    
def schedule_auto_delete():
    global auto_delete_timer
    try:
        print("Cancelling previous timer...")
        auto_delete_timer.cancel()
    except NameError:
        print("Timer hasn't been yet defined.")
        print(f"Scheduling the automatic deletion in {auto_delete_hours} hours.")
    else:
        print("Timer is cancelled.")
        print(f"Re-scheduling the automatic deletion in {auto_delete_hours} hours.")
    auto_delete_timer = threading.Timer(auto_delete_hours * 60 * 60, delete_endpoint)
    auto_delete_timer.start()    

print("Checking notebook for automated run...")
print(f"Seconds since last cell execution: {seconds_since_last_cell_timestamp}")
if seconds_since_last_cell_timestamp > grace_period_seconds:
    print("Cell is executed manually. Deleting endpoint.")
    delete_endpoint()
else:
    print("Cell is executed automatically. Skipping endpoint delete. Don't forget to run the cell again to delete endpoint manually.")
    schedule_auto_delete()

If you don't want to delete the endpoint right now, but want to extend the automatic deletion instead, run the below cell manually. The countdown for automatic deletion will start over.

In [ ]:
seconds_since_last_cell_timestamp = int(time.time() - last_cell_timestamp)
print(f"Seconds since last cell execution: {seconds_since_last_cell_timestamp}")

if seconds_since_last_cell_timestamp > grace_period_seconds:
    print(f"Cell is executed manually. Re-scheduling the timer.")
    schedule_auto_delete()
else:
    print("Cell is executed automatically. Skipping re-scheduling. Run the cell again to re-schedule deletion.")